In [ ]:
#pip install -U predibase

Note: you may need to restart the kernel to use updated packages.


In [41]:
from predibase import Predibase, FinetuningConfig, DeploymentConfig

pb = Predibase(api_token = "pb_-21w1fmTXC6H-WWDumT7mA")

Connected to Predibase as User(id=bb77bdda-4f97-4faa-a128-c549c8d64770, username=rifahathjavid1999@gmail.com)

In [ ]:
lorax_client = pb.deployments.client("zephyr-7b-beta")

In [ ]:
from datasets import load_dataset

dataset = load_dataset("Victorano/customer-support-training-dataset", trust_remote_code=True)
train_data = dataset['train']
train_data

Dataset({
    features: ['flags', 'instruction', 'category', 'intent', 'response'],
    num_rows: 26872
})

In [ ]:
import json


output_path = "zephyr_dataset.jsonl"

with open(output_path, "w") as f:
    for sample in train_data:
        prompt = f"<|user|>\\n {sample['instruction']} </s>\\n<|assistant|>\\n"
        completion = sample['response'].strip()
        json.dump({"prompt": prompt, "completion": completion}, f)
        f.write("\n")

print(f"Saved dataset to {output_path}")

Saved dataset to zephyr_dataset.jsonl


In [ ]:
#pip install -q transformers -U

Note: you may need to restart the kernel to use updated packages.


In [46]:
repo = pb.repos.create(
    name="customer-support",
    description="A Zephyr-7B-Beta fine-tuned model that provides polite and helpful responses to customer support queries.",
    exists_ok=True
)


In [58]:
dataset = pb.datasets.from_file("zephyr_dataset.jsonl", name="zephyr_dataset")
#dataset = pb.upload_file('zephyr_dataset.jsonl', 'zephyr_dataset')

In [47]:
config = FinetuningConfig(
    base_model="Zephyr-7B-Beta",
    epochs=3,
    rank=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    learning_rate=0.0002,
)  # Remove the trailing comma




In [59]:
adapter = pb.adapters.create(
    config=config,
    dataset=dataset,
    repo=repo,
    description="Adapter fine-tuning of Zephyr-7B-Beta for polite, helpful customer support chat.",
)

Successfully requested finetuning of Zephyr-7B-Beta from base as `customer-support/1`. (Job UUID: 8383d609-de28-47a5-bf20-6dc42e5879cb).

Watching progress of finetuning job 8383d609-de28-47a5-bf20-6dc42e5879cb. This call will block until the job has finished. Canceling or terminating this call will NOT cancel or terminate the job itself.

Job is starting. Total queue time: 0:09:06         
Waiting to receive training metrics...

┌────────────┬────────────┬─────────────────┐
│ checkpoint │ train_loss │ validation_loss │
├────────────┼────────────┼─────────────────┤
│     1      │   0.5504   │        --       │
│     2      │   0.5296   │        --       │
│     3      │   0.4842   │        --       │
│     4      │   0.4950   │        --       │
│     5      │   0.5175   │        --       │
│     6      │   0.5011   │        --       │
│     7      │   0.4528   │        --       │
│     8      │   0.4571   │        --       │
│ 848/1260 steps ■■■■■■■■■■■■■■■■■■□□□□□□□□ │

keepalive ping failed
TimeoutError: timed out while closing connection

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/nvcdev01/Documents/fine tune -2/venv/lib/python3.12/site-packages/websockets/sync/connection.py", line 777, in keepalive
    with self.send_context():
         ^^^^^^^^^^^^^^^^^^^
  File "/Users/nvcdev01/Documents/fine tune -2/venv/lib/python3.12/contextlib.py", line 144, in __exit__
    next(self.gen)
  File "/Users/nvcdev01/Documents/fine tune -2/venv/lib/python3.12/site-packages/websockets/sync/connection.py", line 1012, in send_context
    raise self.protocol.close_exc from original_exc
websockets.exceptions.ConnectionClosedError: sent 1011 (internal error) keepalive ping timeout; no close frame received


│     9      │   0.4451   │        --       │
│     10     │   0.4655   │        --       │
│     11     │   0.4172   │        --       │
│     12     │   0.4212   │        --       │
│     13     │   0.4022   │        --       │
└────────────┴────────────┴─────────────────┘
